In [1]:
import pandas as pd
import os
import shutil
from Bio import SeqIO
from itertools import combinations
import subprocess

In [2]:
work_path='/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step2_Cal_HGT/'
ARG_name='blaTEM-1B'
inputname='China_Res_on_MGE'

In [3]:
if os.path.exists(work_path+'Extract'):
    Extract_fasta_file=work_path+'Extract/'
else:
    os.mkdir(work_path+'Extract/')
    Extract_fasta_file=work_path+'Extract/'


if os.path.exists(work_path+'Motifi_Extract_Fasta'):
    Motifi=work_path+'Motifi_Extract_Fasta/'
else:
    os.mkdir(work_path+'Motifi_Extract_Fasta/')
    Motifi=work_path+'Motifi_Extract_Fasta/'


if os.path.exists(work_path+ARG_name+'_list'):
    List_path=work_path+ARG_name+'_list/'
else:
    os.mkdir(work_path+ARG_name+'_list/')
    List_path=work_path+ARG_name+'_list/'


if os.path.exists(work_path+'ARG_file/'):
    file_path=work_path+'ARG_file/'
else:
    os.mkdir(work_path+'ARG_file/')
    file_path=work_path+'ARG_file/'


if os.path.exists(work_path+'fasta_file/'):
    fasta_path=work_path+'fasta_file/'
else:
    os.mkdir(work_path+'fasta_file/')
    fasta_path=work_path+'fasta_file/'

inputname=work_path+inputname+'/'   

In [4]:
def modify_contig_names(input_file, output_file, prefix):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            if line.startswith('>'):
                new_contig_name = f">{prefix}\n"
                f_out.write(new_contig_name)
            else:
                f_out.write(line)

def merge_fasta_files(input_folder, output_file):
    with open(output_file, 'w') as out_handle:
        for filename in os.listdir(input_folder):
            if filename.endswith('.fasta'):
                file_path = os.path.join(input_folder, filename)
                for record in SeqIO.parse(file_path, 'fasta'):
                    SeqIO.write(record, out_handle, 'fasta')

#定义函数，用于截取反向互补
def rev(seq):
    base_trans = {'A':'T', 'C':'G', 'T':'A', 'G':'C', 'a':'t', 'c':'g', 't':'a', 'g':'c'}
    rev_seq = list(reversed(seq))
    rev_seq_list = [base_trans[k] for k in rev_seq]
    rev_seq = ''.join(rev_seq_list)
    return(rev_seq)

########################
def extract_fasta(input, list, output):
    seq_file = {}
    with open(input, 'r') as input_fasta:
        for line in input_fasta:
            line = line.strip()
            if line[0] == '>':
                seq_id = line.split()[0]
                seq_file[seq_id] = ''
            else:
                seq_file[seq_id] += line
    input_fasta.close()

    list_dict = {}
    with open(list, 'r') as list_file:
        for line in list_file:
            if line.strip():
                line = line.strip().split('\t')
                list_dict[line[0]] = [line[1], int(line[2]) - 1, int(line[3]), line[4]]
    list_file.close()

    output_fasta = open(output, 'w')
    for key,value in list_dict.items():
        print('>' + key, file = output_fasta)
        seq = seq_file['>' + value[0]][value[1]:value[2]]
        if value[3] == '+':
            print(seq, file = output_fasta)
        elif value[3] == '-':
            seq = rev(seq)
            print(seq, file = output_fasta)
    output_fasta.close()
    
##########################
    
def remove_duplicates(strings):
    seen_pairs = set()    #########
    result = []
    for string in strings:
        parts = string.split('>')
        pair = tuple(sorted(parts))
        if pair not in seen_pairs:
            seen_pairs.add(pair)
            result.append(string)
    return result

def remove_duplicates2(strings):
    result = []
    for string in strings:
        parts = string.split('>')
        if len(parts) == 2 and parts[0] != parts[1]:
            result.append(string)
    return result


Process_File

In [5]:
inputname_list=os.listdir(inputname)
inputname_list_fna=[x for x in inputname_list if x.endswith('.fasta')]
inputname_list_txt=[x for x in inputname_list if x.endswith('.csv')]
if len(inputname_list_fna) == len(inputname_list_txt):
    print('Start process '+ str(len(inputname_list_txt)) + ' files' )
for i in inputname_list_txt:
    shutil.copy(inputname+i,file_path)
for i in inputname_list_fna:
    shutil.copy(inputname+i,fasta_path)

Start process 227 files


In [6]:
merged_txt=work_path+'All_txt.txt'
with open(merged_txt, 'w') as outfile:
    for filename in os.listdir(file_path):
        if filename.endswith('.csv'):
            temp_file_path = os.path.join(file_path, filename)
            with open(temp_file_path, 'r') as infile:
                outfile.write(infile.read())
file=pd.read_csv(merged_txt)
file=file[file['PRODUCT']==ARG_name]


file.to_csv(work_path+'Target_ARG_summary.csv',index=False)

file=pd.read_csv('Target_ARG_summary.csv')
file_length=file.shape[0]
os.remove(merged_txt)

Start Extract

In [7]:
counter=0
while counter < file_length:
    txt_out=[]
    temp_file=file[file.index==counter]
    fasta_list_filename=temp_file['#FILE'].to_list()[0]
    
    ARG_name=temp_file['PRODUCT'].to_list()[0]

    SEQUENCE=temp_file['SEQUENCE'].to_list()[0]

    START=temp_file['START'].to_list()[0]
    START=str(START)

    END=temp_file['END'].to_list()[0]
    END=str(END)

    STRAND=temp_file['STRAND'].to_list()[0]

    txt_out.append(SEQUENCE)
    txt_out.append(START)
    txt_out.append(END)
    txt_out.append(STRAND)

    if len(txt_out)==4:
        txt_out=pd.DataFrame(txt_out)
        txt_out=txt_out.T
        txt_out.to_csv(List_path+fasta_list_filename+'_'+str(counter),sep='\t',header=False)
    counter+=1

In [8]:
List_path_file=os.listdir(List_path)
for i in List_path_file:
    last_underscore_index=i.rfind('_')
    real_fasta_name=i[:last_underscore_index]
    print(real_fasta_name)

    print('Extracting '+i)
    input_fasta=fasta_path+real_fasta_name
    list_file=List_path+i
    output=Extract_fasta_file+i
    extract_fasta(input_fasta,list_file,output)

fastas=os.listdir(Extract_fasta_file)
for i in fastas:
    contig_name=i.replace('.fasta','')
    modify_contig_names(Extract_fasta_file+i, Motifi+i,contig_name)

SAL03426.fasta
Extracting SAL03426.fasta_40
SAL02648.fasta
Extracting SAL02648.fasta_154
SAL04098.fasta
Extracting SAL04098.fasta_114
SAL02681.fasta
Extracting SAL02681.fasta_153
SAL02652.fasta
Extracting SAL02652.fasta_3
SAL_PA7723AA.fasta
Extracting SAL_PA7723AA.fasta_123
SAL03413.fasta
Extracting SAL03413.fasta_46
SAL02650.fasta
Extracting SAL02650.fasta_95
SAL03419.fasta
Extracting SAL03419.fasta_47
SAL03307.fasta
Extracting SAL03307.fasta_137
SAL00718.fasta
Extracting SAL00718.fasta_124
SAL02651.fasta
Extracting SAL02651.fasta_139
SAL03593.fasta
Extracting SAL03593.fasta_113
SAL03416.fasta
Extracting SAL03416.fasta_152
SAL03406.fasta
Extracting SAL03406.fasta_87
SAL02670.fasta
Extracting SAL02670.fasta_136
SAL03448.fasta
Extracting SAL03448.fasta_35
SAL03427.fasta
Extracting SAL03427.fasta_79
SAL02677.fasta
Extracting SAL02677.fasta_115
SAL00708.fasta
Extracting SAL00708.fasta_9
SAL03306.fasta
Extracting SAL03306.fasta_94
SAL_PA7729AA.fasta
Extracting SAL_PA7729AA.fasta_155
SAL033

In [9]:
########多一步 Mutil Fasta Files合并########
Motified_fastas=os.listdir(Motifi)
temp_Motified=[]
for i in Motified_fastas:
    last_underscore_index=i.rfind('_')
    temp_temp_Motified=i[:last_underscore_index]
    temp_Motified.append(temp_temp_Motified)

set_temp_Motified=list(set(temp_Motified))
for i in set_temp_Motified:

    temp_count=temp_Motified.count(i)

    if temp_count > 1:
        temp_Motified_file_list=[x for x in Motified_fastas if x.startswith(i)]

        with open(Motifi+i, 'w') as outfile:
            for ii in temp_Motified_file_list:
                with open(Motifi+ii, 'r') as infile:
                    for line in infile:
                        outfile.write(line)
    else:
        temp_Motified_file_list=[x for x in Motified_fastas if x.startswith(i)][0]
        os.rename(Motifi+temp_Motified_file_list,Motifi+i)

In [10]:
Motified_fastas=os.listdir(Motifi)

for i in Motified_fastas:
    ii = os.path.join(Motifi, i)
    if os.path.isfile(ii) and not i.endswith(".fasta"):
        os.remove(ii)
######传递参数######
Key_Value=os.listdir(Motifi) 
Key_Value=len(Key_Value)
print(Key_Value, 'Strain carried', ARG_name)

181 Strain carried blaTEM-1B


In [11]:
output_file=ARG_name+' merged.fasta'
merge_fasta_files(Motifi, output_file)

构建Blastn文件结构

In [12]:
if os.path.exists(work_path+'blastn'):
    blastn_path=work_path+'blastn'
else:
    os.mkdir(work_path+'blastn')
    blastn_path=work_path+'blastn'

if os.path.exists(work_path+output_file):
    shutil.move(work_path+output_file,blastn_path)
else:
    print(output_file, " Missing")

if os.path.exists(Motifi):
    shutil.move(Motifi,blastn_path)
else:
    print("Motifi folder Missing")

if os.path.exists(work_path+'blastn.sh'):
    shutil.copy(work_path+'blastn.sh',blastn_path)
else:
    print("Blatn Script Missing")

os.mkdir(blastn_path+'/OUT')
os.mkdir(blastn_path+'/merged_DB')
os.rename(blastn_path+'/'+output_file, blastn_path+'/'+'merged.fasta')
shutil.move(blastn_path+'/merged.fasta',blastn_path+'/merged_DB')

'/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step2_Cal_HGT/blastn/merged_DB/merged.fasta'

构建blastn索引

In [13]:
working_directory=blastn_path+'/merged_DB'
command = ["makeblastdb", "-in", "merged.fasta", "-dbtype", "nucl", "-out", "merged", "-parse_seqids"]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)
if result.returncode == 0:
    print("Blastn DB Run Success")
    print(result.stdout.decode())
else:
    print("Blastn DB Run Failed")
    print(result.stderr.decode())


Blastn DB Run Success


Building a new DB, current time: 10/17/2024 10:44:32
New DB name:   /Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step2_Cal_HGT/blastn/merged_DB/merged
New DB title:  merged.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 181 sequences in 0.00478601 seconds.





运行Blastn

In [14]:
working_directory=blastn_path
command = ["sh", "blastn.sh"]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)

# 输出命令执行结果
if result.returncode == 0:
    print("Blastn Run Complete")
    print(result.stdout.decode())
else:
    print("Blastn Run Failed")
    print(result.stderr.decode())

Blastn Run Complete



计算HGT Frequent Percentage

In [15]:
working_directory=blastn_path+'/OUT'
command = ["cat", "*.txt", ">", "Final_blastn_merged.txt"]
result = subprocess.run(" ".join(command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd=working_directory)

if result.returncode == 0:
    print("File Combined")
    print(result.stdout.decode())
else:
    print("File Combined Failed")
    print(result.stderr.decode())

shutil.move(blastn_path+'/OUT/Final_blastn_merged.txt',work_path)

File Combined



'/Users/jiachenghao/projects/Gal/eLife初稿/eLife_修回/Question4/Step2_Cal_HGT/Final_blastn_merged.txt'

######下面重写

In [16]:
file=pd.read_csv('Final_blastn_merged.txt',sep='\t',header=None)
column=['name1','name2','name3','name4','name5','name6','name7','name8','name9','name10',
        'name11','name12','name13','name14']
file.columns=column
file_length2=file.shape[0]
x_col=['>']*file_length2
file['x']=x_col
file['y']=file.apply(lambda row: ''.join(row[['name1', 'x', 'name2']]), axis=1)

file_name1=list(set(file['name1'].to_list()))
print(len(file_name1), 'Starin carry a total of',len(os.listdir(Extract_fasta_file)),ARG_name)

181 Starin carry a total of 181 blaTEM-1B


In [17]:
All_Y=[]
All_verify=[]
Check_MAX=[]

for i in file_name1:
    temp_file=file[file['name1']==i]

    new_index=i+'>'+i
    temp_file2=temp_file[temp_file['y']==new_index]
    temp_file2_name4=temp_file2['name4'].to_list()
    max_value = max(temp_file2_name4)   #### Coverage == 100%
    Check_MAX.append(max_value)

    temp_file3=temp_file[temp_file['name4']==max_value]
    temp_file3=temp_file3[temp_file3['name3'] ==  100]  #### Identity == 100%
    
    ready_to_depulicate=temp_file3['y'].to_list()
    verfy=temp_file3['name4'].to_list()
    for ii in ready_to_depulicate:
        All_Y.append(ii)
    for iii in verfy:
        All_verify.append(iii)


print('The 100% Coverage is', list(set(Check_MAX))[0])
if len(list(set(Check_MAX))) > 1:
    print('Warning: the 100% Identity is not only.')

The 100% Coverage is 861


In [18]:
filtered_1=remove_duplicates(All_Y)
filtered_2=remove_duplicates2(filtered_1)
temp_out=pd.DataFrame(filtered_2)
temp_out.to_csv('HGT_Detailed.csv',index=False)

In [19]:
############All_Y#############
All_Y_Motified=[]
# test=['GMBC10.085_118_34-GMBC10.027_724_19']
for i in All_Y:
    i=i.split('>')
    i_motifi_1=i[0]
    last_underscore_index=i_motifi_1.rfind('_')
    i_motifi_1=i_motifi_1[:last_underscore_index]

    i_motifi_2=i[-1]
    last_underscore_index=i_motifi_2.rfind('_')
    i_motifi_2=i_motifi_2[:last_underscore_index]

    Motified_i=i_motifi_1+'>'+i_motifi_2
    All_Y_Motified.append(Motified_i)

In [20]:
filtered_1=remove_duplicates(All_Y_Motified)
filtered_2=remove_duplicates2(filtered_1)
temp_out=pd.DataFrame(filtered_2)
temp_out.to_csv('HGT_Strain_Non_Detailed.csv',index=False)

In [21]:
# file_length=len(os.listdir(fasta_path))  ###这个地方可能错误了
file_length=Key_Value

In [22]:
# final_all_all=file_length*file_length    #######Bug2
final_all_count=len(filtered_2)
print(final_all_count)
final_all_all=(1+(file_length-1))*((file_length-1)/2)
print(final_all_all) 
HGT_Frequency=final_all_count/final_all_all
HGT_Frequency=round(HGT_Frequency,2)
print('HGT Frequency for ', ARG_name, 'is:', HGT_Frequency)

16290
16290.0
HGT Frequency for  blaTEM-1B is: 1.0


Deleting Files

In [24]:
shutil.rmtree(work_path+'blastn')
shutil.rmtree(work_path+'Extract')
shutil.rmtree(work_path+'fasta_file')
shutil.rmtree(work_path+'ARG_file')
shutil.rmtree(work_path+ARG_name+'_list/')
os.remove(work_path+'Final_blastn_merged.txt')
os.remove(work_path+'Target_ARG_summary.csv')
os.remove(work_path+'HGT_Strain_Non_Detailed.csv')
os.remove(work_path+'HGT_Detailed.csv')